In [1]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torchvision import datasets, transforms
from torch.utils.data import DataLoader, Dataset
import copy
import numpy as np
import matplotlib.pyplot as plt
from tqdm import tqdm
import networkx as nx
from torch.nn.utils import parameters_to_vector, vector_to_parameters

In [2]:
# Graph implementation
def generate_graph(cluster_sizes=[100,100], pin=0.5, pout=0.01, seed=0):
    """Generate a random connected graph"""
    probs = np.array([[pin, pout, pout, pout],[pout, pin, pout, pout],[pout, pout, pin, pout],[pout, pout, pout, pin]])
    while True:
        g = nx.stochastic_block_model(cluster_sizes, probs, seed=0)
        if nx.algorithms.components.is_connected(g):
            return g


cluster_sizes = [10, 10, 10, 10]
features_sizes = [8, 7, 6, 5]
pin = 0.5
pout = 0.1
seed = 0
alpha = 1e-2
lamda = 0#1e-3#1e-3
eta = 1e-2
d0 = min(features_sizes)
no_users = sum(cluster_sizes)
batch_size = 10
epochs = 1
it = 2000
G = generate_graph(cluster_sizes, pin, pout, seed)

# Set a random seed for reproducibility
seed = 17
torch.manual_seed(seed)
torch.backends.cudnn.deterministic = True
torch.backends.cudnn.benchmark = False
np.random.seed(seed)
#nx.draw(G, with_labels=True, node_size=100, alpha=1, linewidths=10)
#plt.show()

In [3]:
# Metropolis weights 
number_nodes = G.number_of_nodes()
weights = np.zeros([number_nodes, number_nodes])
for edge in G.edges():
  i, j = edge[0], edge[1]
  weights[i - 1][j - 1] = 1 / (1 + np.max([G.degree(i), G.degree(j)]))
  weights[j - 1][i - 1] = weights[i - 1][j - 1]

print(weights)

weights = weights + np.diag(1 - np.sum(weights, axis=0))

metropolis_weights = weights
print(metropolis_weights)


[[0.         0.         0.         ... 0.09090909 0.         0.        ]
 [0.         0.         0.         ... 0.         0.         0.        ]
 [0.         0.         0.         ... 0.         0.         0.1       ]
 ...
 [0.09090909 0.         0.         ... 0.         0.         0.        ]
 [0.         0.         0.         ... 0.         0.         0.        ]
 [0.         0.         0.1        ... 0.         0.         0.        ]]
[[0.40353535 0.         0.         ... 0.09090909 0.         0.        ]
 [0.         0.53216783 0.         ... 0.         0.         0.        ]
 [0.         0.         0.13216783 ... 0.         0.         0.1       ]
 ...
 [0.09090909 0.         0.         ... 0.11796537 0.         0.        ]
 [0.         0.         0.         ... 0.         0.18100233 0.        ]
 [0.         0.         0.1        ... 0.         0.         0.1488345 ]]


In [4]:
def degrees(A):
    """Return the degrees of each node of a graph from its adjacency matrix"""
    return np.sum(A, axis=0).reshape(A.shape[0], 1)

def node_degree(n, G):
    cnt = 0
    for i in G.neighbors(n):
        cnt += 1
    return cnt

def get_neighbors(n, G):
    neighbors_list = []
    for i in G.neighbors(n):
        neighbors_list.append(int(i))
    return neighbors_list

In [5]:
datapoints = {}
count = 0
W1 =  np.array([2.0, 2.0, 0.0, 5.0, -2.0, -3.0, 5.0, 2.0])
W2 = np.array([2.0, 2.0, 0.0, 5.0, 2.0, -3.0, -5.0])
W3 =  np.array([2.0, 2.0, 0.0, 5.0, -2.0, -3.0])
W4 =  np.array([2.0, 2.0, 0.0, -5.0, -2.0])
W = [W1, W2, W3, W4]
m = 50
n = [8, 7, 6, 5]
noise_sd = [0.001,0.01,0.1,1.0]

for i, cluster_size in enumerate(cluster_sizes):
    for j in range(cluster_size):
        features = np.random.normal(loc=0.0, scale=1.0, size=(m, n[i]))
        label = np.dot(features, W[i]) + np.random.normal(0,noise_sd[2])
        test_features = np.random.normal(loc=0.0, scale=1.0, size=(100, n[i]))
        test_label = np.dot(test_features, W[i]) + np.random.normal(0,0.05)
        datapoints[count] = {
                'features': features,
                'degree': node_degree(count, G),
                'label': label,
                'neighbors': get_neighbors(count, G),
                'exact_weights': torch.from_numpy(W[i]),
                'input_size': n[i],
                'test_features':test_features,
                'test_labels': test_label
            }
        count += 1

In [6]:
class MyDataset(Dataset):
    def __init__(self, data, targets, transform=None):
        self.data = torch.FloatTensor(data)
        self.targets = torch.FloatTensor(targets).unsqueeze(-1)
        
    def __getitem__(self, index):
        x = self.data[index]
        y = self.targets[index]

        return x, y
    
    def __len__(self):
        return len(self.data)


In [7]:
class MLP_Net(nn.Module):
    def __init__(self, input_size, user_id):
        super(MLP_Net, self).__init__()
        self.fc1 = nn.Linear(input_size, 1, bias=False)
        #self.fc2 = nn.Linear(4, 1, bias=False)
        #self.fc3 = nn.Linear(200, 10)
        self.user_id = user_id

    def forward(self, x):
        x = torch.flatten(x, 1)
        #x = F.relu(self.fc1(x))
        output = self.fc1(x)
        #output = self.fc3(x)
        return output

In [8]:
from typing import Iterable, Optional

def grads_to_vector(parameters: Iterable[torch.Tensor]) -> torch.Tensor:
    r"""Convert parameters to one vector

    Args:
        parameters (Iterable[Tensor]): an iterator of Tensors that are the
            parameters of a model.

    Returns:
        The parameters represented by a single vector
    """
    # Flag for the device where the parameter is located
    param_device = None

    vec = []
    for param in parameters:
        # Ensure the parameters are located in the same device
        param_device = param.grad

        vec.append(param_device.view(-1))
    return torch.cat(vec)

In [9]:
model = MLP_Net(datapoints[19]["input_size"], user_id=0)

lr = 0.01

dataloader = DataLoader(MyDataset(datapoints[19]["features"], datapoints[19]["label"]), batch_size=20, shuffle=True)
optimizer = torch.optim.SGD(model.parameters(), lr=0.01)
for i in range(100):
    for j, (x, y) in zip(range(1), dataloader):
        criterion = nn.MSELoss()
        optimizer.zero_grad()
        yhat = model(x)

        loss = criterion(yhat, y)
        
        loss.backward()
        print(i, loss.detach(), parameters_to_vector(model.parameters()).detach())
        #optimizer.step()
        new_model = parameters_to_vector(model.parameters()) - lr * grads_to_vector(model.parameters())
        vector_to_parameters(parameters=model.parameters(), vec=new_model)
        #if i % 50 ==0:
            #lr *= 0.9
            

#parameters_to_vector(model.parameters())

0 tensor(66.9203) tensor([-0.0497,  0.0265,  0.2496, -0.2843, -0.3558,  0.0373, -0.0888])
1 tensor(88.9725) tensor([-0.0605,  0.0263,  0.2387, -0.1926, -0.3544, -0.0526, -0.2111])
2 tensor(75.7507) tensor([-0.0070,  0.0675,  0.1969, -0.0852, -0.4048, -0.1659, -0.3788])
3 tensor(57.8489) tensor([ 0.0688,  0.0720,  0.1347,  0.0082, -0.4030, -0.2452, -0.5140])
4 tensor(41.5260) tensor([ 0.1294,  0.0949,  0.1483,  0.0863, -0.4131, -0.3008, -0.6203])
5 tensor(54.2266) tensor([ 0.1882,  0.1076,  0.1548,  0.1297, -0.3938, -0.3703, -0.6778])
6 tensor(51.4614) tensor([ 0.2372,  0.1393,  0.1194,  0.2144, -0.4359, -0.4483, -0.7727])
7 tensor(69.9925) tensor([ 0.2778,  0.1552,  0.0620,  0.2935, -0.4229, -0.4940, -0.8651])
8 tensor(44.3733) tensor([ 0.2862,  0.1883,  0.0663,  0.4655, -0.4219, -0.5672, -0.9444])
9 tensor(36.1280) tensor([ 0.3150,  0.1833,  0.0273,  0.5475, -0.4001, -0.6124, -1.0187])
10 tensor(39.9554) tensor([ 0.3555,  0.2012, -0.0044,  0.5797, -0.3986, -0.6658, -1.1053])
11 tensor

In [10]:
parameters_to_vector(model.parameters())

tensor([ 1.6380,  1.5253, -0.3111,  4.0060,  0.5291, -2.7091, -4.2288],
       grad_fn=<CatBackward0>)

In [11]:
class CNN_Net(nn.Module):
    def __init__(self):
        super(CNN_Net, self).__init__()
        self.conv1 = nn.Conv2d(1, 32, kernel_size=5)
        self.conv2 = nn.Conv2d(32, 64, kernel_size=5)
        self.pool = nn.MaxPool2d(2,2)
        self.dropout = nn.Dropout(p=0.2)
        self.fc1 = nn.Linear(1024, 512)
        self.fc2 = nn.Linear(512, 10)

    def forward(self, x):
        x = self.pool(F.relu(self.conv1(x)))
        x = self.pool(F.relu(self.conv2(x)))
        x = self.dropout(x)
        x = torch.flatten(x, 1)
        x = F.relu(self.fc1(x))
        output = self.fc2(x)
        return output

In [12]:
class ClientUpdate(object):
    def __init__(self, dataset, batchSize, alpha, lamda, epochs, projection_list, projected_weights):
        self.train_loader = DataLoader(MyDataset(dataset["features"], dataset["label"]), batch_size=batchSize, shuffle=True)
        #self.learning_rate = learning_rate
        self.epochs = epochs
        self.batchSize = batchSize

    def train(self, model):
        criterion = nn.MSELoss()
        optimizer = torch.optim.SGD(model.parameters(), lr=1e-3, momentum=0.5)

        e_loss = []
        for epoch in range(1, self.epochs+1):
            train_loss = 0
            model.train()
            for i, (data, labels) in zip(range(1), self.train_loader):
                data, labels = data, labels
                optimizer.zero_grad() 
                output = model(data)  
                loss = criterion(output, labels)
                #loss += mu/2 * torch.norm(client_param.data - server_param.data)**2
                loss.backward()
                grads = grads_to_vector(model.parameters())
                #optimizer.step()
                train_loss += loss.item()*data.size(0)
                weights = parameters_to_vector(model.parameters())
                mat_vec_sum = torch.zeros_like(weights)
                for j in G.neighbors(model.user_id):
                    mat_vec_sum = torch.add(mat_vec_sum, torch.matmul(torch.transpose(projection_list[model.user_id][j], 0, 1), 
                                                         projected_weights[j][model.user_id] - projected_weights[model.user_id][j]))
                
                model_update = parameters_to_vector(model.parameters()) - alpha * (grads + lamda * mat_vec_sum)
                
            vector_to_parameters(parameters=model.parameters(), vec=model_update)
                

            train_loss = train_loss/self.batchSize#len(self.train_loader.dataset) 
            e_loss.append(train_loss)

        total_loss = e_loss#sum(e_loss)/len(e_loss)

        return model.state_dict(), total_loss

In [13]:
# Preparing projection matrices
models = [MLP_Net(input_size=datapoints[i]['input_size'], user_id=i) for i in range(no_users)]
#temp = MLP_Net()
projection_list = []
projected_weights = []

def update_ProjWeight(projection_list, projected_weights, models, first_run=True):
    for i in range(no_users):
        neighbors_mat = []
        neighbors_weights = []
        for j in range(no_users):
            if j in G.neighbors(i):
                with torch.no_grad():
                    if first_run == True:
                        # Specify the dimensions of the rectangular matrix
                        row, column = d0, parameters_to_vector(models[i].parameters()).size()[0]

                        # Generate random values for the diagonal from a normal distribution
                        diag_values = 1.0 + 1.0 * torch.randn(min(row, column))

                        # Create a rectangular matrix with diagonal elements
                        mat = torch.diag(diag_values)

                        # If the matrix is larger than the diagonal vector, fill the remaining elements with zeros
                       
                        mat = torch.cat((mat, torch.zeros(row, column - row)), dim=1)

                        

                        # Append the matrix to the list
                        neighbors_mat.append(mat)
                        neighbors_weights.append(torch.matmul(mat, parameters_to_vector(models[i].parameters())))
                    else:
                        neighbors_weights.append(torch.matmul(projection_list[i][j], parameters_to_vector(models[i].parameters())))
            else:
                neighbors_mat.append(0)
                neighbors_weights.append(0)
        if first_run == True:
            projection_list.append(neighbors_mat)
        projected_weights.append(neighbors_weights)

update_ProjWeight(projection_list, projected_weights, models)



In [14]:
def testing(model, dataset, bs, criterion): 
    test_loss = 0
    correct = 0
    test_loader = DataLoader(MyDataset(dataset["test_features"], dataset["test_labels"]), batch_size=bs)
    l = len(test_loader)
    model.eval()
    for data, labels in test_loader:
        data, labels = data, labels
        output = model(data)
        loss = criterion(output, labels)
        test_loss += loss.item()*data.size(0)
        #_, pred = torch.max(output, 1)
        #correct += pred.eq(labels.data.view_as(pred)).sum().item()
    
    test_loss /= len(test_loader.dataset)
    
    return test_loss

In [15]:
def rel_error(model):
    return (torch.norm(parameters_to_vector(model.parameters()) - datapoints[model.user_id]['exact_weights']) / torch.norm(datapoints[model.user_id]['exact_weights'])).detach()

In [16]:
torch.norm(datapoints[model.user_id]['exact_weights'])

tensor(8.6603, dtype=torch.float64)

In [17]:
#global_model = CNN_Net().cuda()
models = [MLP_Net(input_size=datapoints[i]['input_size'], user_id=i) for i in range(no_users)]
dummy_models = [MLP_Net(input_size=datapoints[i]['input_size'], user_id=i) for i in range(no_users)]

#model.load_state_dict(global_model.state_dict())

criterion = nn.MSELoss()


train_loss = []
test_loss = []
test_accuracy = []
total_rel_error = []

in_cluster_proj_norm = []
out_cluster_proj_norm = []
in_cluster_proj_diff_norm = []
out_cluster_proj_diff_norm = []
task_loss = {'0':[],
                '1':[],
                '2':[],
                '3':[]}
task_rel_error = {'0':[],
            '1':[],
            '2':[],
            '3':[]}

for curr_round in tqdm(range(1, it+1)):
    w, local_loss = [], []

    
    for i in range(no_users):
        dummy_models[i].load_state_dict(models[i].state_dict())
        local_update = ClientUpdate(dataset=datapoints[i], batchSize=batch_size, alpha=alpha, lamda=lamda, epochs=1, projection_list=projection_list, projected_weights=projected_weights)
        weights, loss = local_update.train(dummy_models[i])
        w.append(weights)
        local_loss.append(loss)
        models[i].load_state_dict(w[i])
        
    
    
    # Update prjection matrix
    projected_weights = []
    update_ProjWeight(projection_list, projected_weights, models, first_run=False)
    
    #print(projection_list[0], projected_weights[0])
    
    for i in range(no_users):
        weights = parameters_to_vector(models[i].parameters())
        for j in G.neighbors(i):
            temp_mat = torch.outer(projected_weights[i][j] - projected_weights[j][i], weights).clone()
            projection_list[i][j] = torch.add(projection_list[i][j], -1 * eta * lamda * temp_mat)
    
    in_cluster_proj_norm_round = 0
    out_cluster_proj_norm_round = 0
    in_cluster_proj_diff_round = 0
    out_cluster_proj_diff_round = 0
    in_edges = 0
    out_edges = 0
    
    for i in range(no_users//4):
        for j in G.neighbors(i):
            if j < no_users//4:
                in_edges += 1
                in_cluster_proj_norm_round += torch.norm(projection_list[i][j]).detach().numpy()
                in_cluster_proj_diff_round += torch.norm(projected_weights[i][j] - projected_weights[j][i]).detach().numpy()
            else:
                out_edges += 1
                out_cluster_proj_norm_round += torch.norm(projection_list[i][j]).detach().numpy()
                out_cluster_proj_diff_round += torch.norm(projected_weights[i][j] - projected_weights[j][i]).detach().numpy()
    in_cluster_proj_norm.append(in_cluster_proj_norm_round / in_edges)
    out_cluster_proj_norm.append(out_cluster_proj_norm_round / out_edges)
    in_cluster_proj_diff_norm.append(in_cluster_proj_diff_round / in_edges)
    out_cluster_proj_diff_norm.append(out_cluster_proj_diff_round / out_edges)
            
                                         
                                              
    
        
        
        
    
    




          
            

    local_test_acc = []
    local_test_loss = []
    user_rel_error = 0
    per_task_loss = []
    per_task_rel_error = []
    
    for k in range(no_users):
      
        g_loss = testing(models[i], datapoints[i], 50, criterion)
        local_test_loss.append(g_loss)
        user_rel_error += rel_error(models[i])
        if (k + 1) % 10 == 0:
            task_loss[str(k // 10)].append(sum(per_task_loss) / 10)
            task_rel_error[str(k // 10)].append(sum(per_task_rel_error) / 10)
            per_task_loss = []
            per_task_rel_error = []
        per_task_loss.append(g_loss)
        per_task_rel_error.append(rel_error(models[i]))
    
    
        

    g_loss = sum(local_test_loss) / len(local_test_loss)
    total_rel_error.append(user_rel_error / no_users)
    
    

    test_loss.append(g_loss)
    #test_accuracy.append(g_accuracy)
    print("Training_loss %2.5f,   Relative Error %2.5f"% (test_loss[-1], total_rel_error[-1]))

  0%|          | 2/2000 [00:00<03:52,  8.61it/s]

Training_loss 70.15044,   Relative Error 0.92818
Training_loss 67.39997,   Relative Error 0.91148


  0%|          | 4/2000 [00:00<03:51,  8.62it/s]

Training_loss 63.95627,   Relative Error 0.88850
Training_loss 60.72331,   Relative Error 0.86573


  0%|          | 6/2000 [00:00<04:05,  8.12it/s]

...

Training_loss 0.00084,   Relative Error 0.00346
Training_loss 0.00077,   Relative Error 0.00330


100%|█████████▉| 1999/2000 [05:51<00:00,  5.74it/s]

Training_loss 0.00069,   Relative Error 0.00313
Training_loss 0.00070,   Relative Error 0.00314


100%|██████████| 2000/2000 [05:51<00:00,  5.69it/s]

Training_loss 0.00068,   Relative Error 0.00309


In [18]:
#plot.plot(test_loss)
print(parameters_to_vector(models[19].parameters()), W[1])

tensor([ 1.9913,  2.0138,  0.0238,  5.0157,  1.9681, -2.9953, -5.0207],
       grad_fn=<CatBackward0>) [ 2.  2.  0.  5.  2. -3. -5.]


In [19]:
for i in G.neighbors(3):
    print(i)

0
4
5
6
15
16
19
25
31


In [20]:
test_loss = np.array(test_loss)
total_rel_error = np.array(total_rel_error)

in_cluster_proj_norm = np.array(in_cluster_proj_norm)
out_cluster_proj_norm = np.array(out_cluster_proj_norm)
in_cluster_proj_diff_norm = np.array(in_cluster_proj_diff_norm)
out_cluster_proj_diff_norm = np.array(out_cluster_proj_diff_norm)


In [21]:
np.save( 'training_loss_sheave_fml' + str(lamda).replace('.', '_')+ '_pout' + str(pout).replace('.', '_'), test_loss)
np.save('relative_error_sheave_fml' + str(lamda).replace('.', '_')+ '_pout' + str(pout).replace('.', '_'), total_rel_error)
np.save( 'in_cluster_proj_norm_sheave_fml' + str(lamda).replace('.', '_')+ '_pout' + str(pout).replace('.', '_'), in_cluster_proj_norm)
np.save('out_cluster_proj_norm_sheave_fml' + str(lamda).replace('.', '_')+ '_pout' + str(pout).replace('.', '_'), out_cluster_proj_norm)
np.save( 'in_cluster_proj_diff_norm_sheave_fml' + str(lamda).replace('.', '_')+ '_pout' + str(pout).replace('.', '_'), in_cluster_proj_diff_norm)
np.save('out_cluster_proj_diff_norm_sheave_fml' + str(lamda).replace('.', '_')+ '_pout' + str(pout).replace('.', '_'), out_cluster_proj_diff_norm)

In [22]:
'training_loss_sheave_fml' + str(lamda).replace('.', '_')+ '_pout' + str(pout).replace('.', '_')

'training_loss_sheave_fml0_pout0_1'

In [23]:
'relative_error_sheave_fml' + str(lamda).replace('.', '_')+ '_pout' + str(pout).replace('.', '_')

'relative_error_sheave_fml0_pout0_1'

In [24]:
'in_cluster_proj_norm_sheave_fml' + str(lamda).replace('.', '_')+ '_pout' + str(pout).replace('.', '_')

'in_cluster_proj_norm_sheave_fml0_pout0_1'

In [25]:
'out_cluster_proj_norm_sheave_fml' + str(lamda).replace('.', '_')+ '_pout' + str(pout).replace('.', '_')

'out_cluster_proj_norm_sheave_fml0_pout0_1'

In [26]:
'in_cluster_proj_diff_norm_sheave_fml' + str(lamda).replace('.', '_')+ '_pout' + str(pout).replace('.', '_')

'in_cluster_proj_diff_norm_sheave_fml0_pout0_1'

In [27]:
'out_cluster_proj_diff_norm_sheave_fml' + str(lamda).replace('.', '_')+ '_pout' + str(pout).replace('.', '_')

'out_cluster_proj_diff_norm_sheave_fml0_pout0_1'

In [28]:
# task_loss dictionary
for key, value in task_loss.items():
    # Convert the list to a NumPy array
    array_loss = np.array(value)
    
    # Save the NumPy array using the specified format
    np.save('training_loss_sheave_fml_task' + key + '_' + str(lamda).replace('.', '_') + '_pout' + str(pout).replace('.', '_'), array_loss)

# task_rel_error dictionary
for key, value in task_rel_error.items():
    # Convert the list to a NumPy array
    array_rel_error = np.array(value)
    
    # Save the NumPy array using the specified format
    np.save('relative_error_sheave_fml_task' + key + '_' + str(lamda).replace('.', '_') + '_pout' + str(pout).replace('.', '_'), array_rel_error)

In [29]:
'relative_error_sheave_fml_task' + key + '_' + str(lamda).replace('.', '_') + '_pout' + str(pout).replace('.', '_')

'relative_error_sheave_fml_task3_0_pout0_1'

In [30]:
task_loss['0']

[63.13539276123047,
 60.65997734069824,
 57.56063976287842,
 54.65097942352295,
 52.76997756958008,
 51.81614799499512,
 51.20059776306152,
 49.64694728851318,
 46.08466129302978,
 44.88493022918701,
 44.09264945983887,
 42.73548259735107,
 41.439816856384276,
 39.06672878265381,
 37.98995876312256,
 36.43961448669434,
 33.99563026428223,
 33.292541313171384,
 32.019747734069824,
 30.53561496734619,
 28.722781562805174,
 27.2619647026062,
 26.236892223358154,
 24.96845111846924,
 24.655785369873048,
 24.419215393066406,
 23.646046543121336,
 22.569285106658935,
 21.432673931121826,
 20.54486789703369,
 19.926401996612547,
 19.081475257873535,
 18.27013235092163,
 17.656451511383057,
 17.356765937805175,
 16.286538791656493,
 15.80983943939209,
 15.4324161529541,
 14.980264377593993,
 14.458524942398071,
 14.087833356857299,
 13.744899415969849,
 13.526876592636109,
 13.087453508377076,
 12.432331895828247,
 12.239828252792359,
 12.025924444198608,
 11.763131046295166,
 11.3741592407226

In [31]:
task_loss['1']

[70.15043640136719,
 67.39997482299805,
 63.95626640319824,
 60.723310470581055,
 58.63330841064453,
 57.5734977722168,
 56.88955307006836,
 55.16327476501465,
 51.20517921447754,
 49.87214469909668,
 48.9918327331543,
 47.483869552612305,
 46.044240951538086,
 43.4074764251709,
 42.2110652923584,
 40.488460540771484,
 37.77292251586914,
 36.99171257019043,
 35.577497482299805,
 33.9284610748291,
 31.914201736450195,
 30.291071891784668,
 29.15210247039795,
 27.74272346496582,
 27.39531707763672,
 27.132461547851562,
 26.273385047912598,
 25.07698345184326,
 23.814082145690918,
 22.8276309967041,
 22.140446662902832,
 21.20163917541504,
 20.30014705657959,
 19.618279457092285,
 19.285295486450195,
 18.09615421295166,
 17.56648826599121,
 17.14712905883789,
 16.64473819732666,
 16.065027713775635,
 15.653148174285889,
 15.27211046218872,
 15.029862880706787,
 14.541615009307861,
 13.81370210647583,
 13.599809169769287,
 13.362138271331787,
 13.070145606994629,
 12.637954711914062,
 12.4

In [32]:
projection_list[0]

[0,
 0,
 0,
 tensor([[1.9701, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000],
         [0.0000, 1.5921, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000],
         [0.0000, 0.0000, 0.0033, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000],
         [0.0000, 0.0000, 0.0000, 0.1557, 0.0000, 0.0000, 0.0000, 0.0000],
         [0.0000, 0.0000, 0.0000, 0.0000, 2.1053, 0.0000, 0.0000, 0.0000]],
        grad_fn=<AddBackward0>),
 tensor([[2.0324, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000],
         [0.0000, 1.9731, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000],
         [0.0000, 0.0000, 1.0599, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000],
         [0.0000, 0.0000, 0.0000, 1.7370, 0.0000, 0.0000, 0.0000, 0.0000],
         [0.0000, 0.0000, 0.0000, 0.0000, 1.6656, 0.0000, 0.0000, 0.0000]],
        grad_fn=<AddBackward0>),
 0,
 tensor([[ 2.8000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000],
         [ 0.0000,  1.5508,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000],


In [33]:
parameters_to_vector(models[0])

TypeError: 'MLP_Net' object is not iterable